### 데이터 프레임 다른 포맷 형식으로 저장

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [1]:
import pandas as pd

In [2]:
dict_data = {
    "c0":[1,2,3],
    "c1":[4,5,6]
}
df = pd.DataFrame(dict_data)
df

,c0,c1
0,1,4
1,2,5
2,3,6


In [3]:
df.to_csv("./df1.csv",index=False)  # 인덱스를 저장 안하고 데이터만 저장

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),options=chrome_options
    )
    return driver   # 드라이버 가지고 오는 함수 실행

### Youtube 댓글 크롤링 후 데이터 프레임 생성

In [24]:
from bs4 import BeautifulSoup

In [57]:
import time
from selenium.webdriver.common.by import By   
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

driver = set_chrome_driver()
driver.get("https://www.youtube.com/watch?v=HosW0gulISQ")

time.sleep(5)

# 닫기 버튼
WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]'))).click()


# 현재 문서 높이
prev_height = driver.execute_script("return document.documentElement.scrollHeight")

# 스크롤 이동 시간 간격
interval = 5

while True:

    # 스크롤 이동
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 스크롤이 진행된 후 현재 문서 높이
    curr_height = driver.execute_script("return document.documentElement.scrollHeight")

    if curr_height == prev_height:
        break

    prev_height = curr_height


time.sleep(2)

soup = BeautifulSoup(driver.page_source,"lxml")


In [58]:
# 파싱 - id, 내용

# 작성자 : #author-text > span
# 내용 : #content-text

author_list = soup.select("#author-text > span")
contents = soup.select("#content-text")

print(len(author_list),len(contents))

1265 1265


In [59]:
# 태그 제거

author_new = []
contents_new = []

for i in range(len(author_list)):
    # 작성자 가져오기
    author = author_list[i].text
    author = author.replace("\n","")
    author = author.replace("\t","")
    author = author.replace("    "," ")
    author_new.append(author)

    # 내용 가져오기
    content = contents[i].text
    content = content.replace("\n","")
    content = content.replace("\t","")
    content = content.replace("    "," ")
    contents_new.append(content)

In [60]:
df = pd.DataFrame({"author":author_list, "comment":contents})
df.head()

,author,comment
0,[\n @user-vj4lt5vu2r\n ],"[[""성공한 사람에게는 추억을""], [\n], [""성공할려는 자에게는 용기를""]]"
1,[\n @Jerjeon\n ],[와 진짜 영화 한장면 넣어놓은거 같음..전율이 미쳤음...와...소름돋는다]
2,[\n @user-yz5ti1wp3u\n ],"[[데뷔할 때만 해도 20대였던 크리스와 멤버들..], [\n], [어느덧 벌써 4..."
3,[\n @user-hd8vz7ww7w\n ],"[진짜 들을 때마다 매순간 빠짐없이 벅차오르는 개미친 노래 .., ㄹㅇ 죽기 전에는..."
4,[\n @qjsdurwhk\n ],[10년 넘게 들어도 질리지 않는 명곡...]


In [61]:
df.to_csv("./youtube.csv", index=False)

#### [참고] 전체 스크롤의 크기 확인하기

In [ ]:
# body 태그
height = driver.execute_script("return document.body.scrollHeight")
print("body_scrollHeight", height)

# html 태그
doc_height = driver.execute_script("return document.documentElement.scrollHeight")
print("documentElement_scrollHeight", doc_height)